In [4]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
    AIMessage,
    trim_messages
)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from operator import itemgetter
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.retrieval_qa.base import RetrievalQA
import os

class LanguageModel:
    def __init__(self, model_name="gpt-4o"):
        self.store = {}

        llm_model = ChatOpenAI(model=model_name)

        rag_csv_directory = "rag_csv"
        csv_files = [os.path.join(rag_csv_directory, f) for f in os.listdir(rag_csv_directory) if f.endswith('.csv')]
        ref_docs = []
        for csv_file in csv_files:
            loader = CSVLoader(
                file_path=csv_file,
                csv_args= {
                    "fieldnames": [
                        "Store Link", "Name", "Seller", "Image Links", "Price", "Seller Description"
                    ]
                }
            )
            document = loader.load()
            ref_docs.extend(document)
        embeddings = OpenAIEmbeddings()
        vector_store = FAISS.from_documents(ref_docs, embeddings)
        docs_retriever = vector_store.as_retriever()
        csv_chain = RetrievalQA.from_chain_type(llm=llm_model, retriever=docs_retriever)

        @tool
        def csv_tool(search_term: str) -> str:
            """Useful for answering questions with loaded CSV."""
            return csv_chain.run(search_term)

        tools = [csv_tool]
        self.tool_dict = {
            "csv_tool": csv_tool
        }

        llm_model_with_tools = llm_model.bind_tools(tools)

        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=
            "You are a collectible item curator. "
            "When a user prompts for an item, always look at reference documents first for the item "
            "using csv_tool. If item is found, provide the price, store and image link for each item. "
            "If item is not found, say item is not found. Provide a list of 10 items per query."
            # "If item is not found in reference documents, look up the item online with the search_tool, "
            # 'appending "site:" and the website link to the search term if a website link is provided.'
            # "Include each online source used next to the cited information."
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            MessagesPlaceholder(variable_name="input"),
        ])
        trimmer = trim_messages(
            max_tokens=30,
            strategy="last",
            token_counter=len,
            include_system=True,
            allow_partial=False,
            start_on="human",
        )
        chain = (
            RunnablePassthrough.assign(
                chat_history=itemgetter("chat_history") | trimmer
            )
            | prompt
            | llm_model_with_tools
        )
        self.with_message_history = RunnableWithMessageHistory(
            chain,
            self.get_session_history,
            history_messages_key="chat_history",
            input_messages_key="input",
        )

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    def get_llm_response(
        self, text: str, session_id: str, img_base64: str = ""
    ) -> list[str]:
        gathered_response = []
        config = {
            "configurable": {
                "session_id": session_id
            }
        }
        chat_history = self.get_session_history(session_id)
        # insert AI message to prevent trimmer throwing an error
        # due to empty chat history
        if not chat_history.messages:
            chat_history.add_ai_message("I’m Pompa AI, rare antiques collector.")
        input = [{"type": "text", "text": text}]
        if img_base64:
            input.append({
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{img_base64}"
                }
            })
        response = self.with_message_history.invoke(
            {
                "input": [HumanMessage(content=input)]
            },
            config=config,
        )
        gathered_response.append(response.content)

        # handle when AI determines tool needs to be called
        while response.tool_calls:
            for tool_call in response.tool_calls:
                selected_tool = self.tool_dict[tool_call["name"].lower()]
                tool_msg = selected_tool.invoke(tool_call)
                chat_history.add_message(tool_msg)

            after_tool_response = self.with_message_history.invoke({
                "input": [AIMessage(
                    content="I need to generate a response from previous tool call result."
                )]
            }, config=config)
            gathered_response.append(after_tool_response.content)
            response = after_tool_response

        return gathered_response

pompa_ai = LanguageModel()

In [5]:
responses = pompa_ai.get_llm_response(
    text="Do you have morgan dollars for sale?",
    session_id="test chat"
)
for response in responses:
    print(response)

C:\Users\adria\AppData\Local\Temp\ipykernel_24276\3492935240.py:52: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return csv_chain.run(search_term)



Here are some Morgan Dollars available for sale from eBay sellers:

1. **1900 Morgan Dollar MS64 NGC**
   - Price: $125.00
   - Seller: Park Ave Numismatics
   - [Store Link](https://www.ebay.com/itm/146383131998)
   - ![Image](https://i.ebayimg.com/images/g/aIQAAeSwjcdnq1rA/s-l1600.webp)

2. **1884 Morgan Dollar MS64 PCGS**
   - Price: $329.95
   - Seller: Park Ave Numismatics
   - [Store Link](https://www.ebay.com/itm/135423318346)
   - ![Image](https://i.ebayimg.com/images/g/B80AAOSwMkBnUf9f/s-l1600.webp)

3. **1887 Morgan Dollar MS65 PCGS**
   - Price: $1,099.95
   - Seller: Park Ave Numismatics
   - [Store Link](https://www.ebay.com/itm/146041062448)
   - ![Image](https://i.ebayimg.com/images/g/AzQAAOSwGyVm7ZaV/s-l1600.webp)

4. **1903 Morgan Dollar PCGS MS64 Better Date $1**
   - Price: $185.00
   - Seller: Acousha Collectibles
   - [Store Link](https://www.ebay.com/itm/396204670225)
   - ![Image](https://i.ebayimg.com/images/g/g2UAAOSw9bpnrPeT/s-l1600.webp)

If you're intereste